You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction: 
- Brief overview of the rapid advancements in Artificial Intelligence (AI)
- Importance of staying up-to-date with the latest trends and key players in the AI industry
- Preview of the key points to be discussed in the article

Key Points:
1. Latest Trends in Artificial Intelligence:
- Deep learning and neural networks
- Natural language processing
- AI in healthcare, finance, and other industries
-

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has been rapidly advancing, revolutionizing various industries and shaping the future of technology. Staying up-to-date with the latest trends and key players in the AI industry is crucial for tech-savvy individuals, business professionals, and students interested in AI. In this article, we will explore the latest trends in AI, key players in the industry, and noteworthy news that is shaping the future of AI.

## Latest Trends in Artificial Intelligence

Deep learning and neural networks have been at the forefront of AI innovation, enabling machines to learn and make decisions like humans. Natural language processing has also seen significant advancements, with AI-powered systems understanding and generating human language. AI applications in healthcare, finance, and other industries continue to grow, offering solutions to comp

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction

Artificial Intelligence (AI) has been rapidly advancing, revolutionizing various industries and shaping the future of technology. Staying up-to-date with the latest trends and key players in the AI industry is crucial for tech-savvy individuals, business professionals, and students interested in AI. In this article, we will explore the latest trends in AI, key players in the industry, and noteworthy news that is shaping the future of AI.

## Latest Trends in Artificial Intelligence

Deep learning and neural networks have been at the forefront of AI innovation, enabling machines to learn and make decisions like humans. Natural language processing has also seen significant advancements, with AI-powered systems understanding and generating human language. AI applications in healthcare, finance, and other industries continue to grow, offering solutions to complex problems. However, ethical considerations in AI development, such as bias in algorithms and data privacy, remain crucial in ensuring responsible AI implementation.

## Key Players in the AI Industry

Major tech companies like Google (Google Brain, DeepMind), Amazon (Alexa, Amazon Web Services), Microsoft (Cortana, Azure), and IBM (Watson) have been leading the way in AI research and development. Alongside these tech giants, startups disrupting the AI space with innovative solutions and products are also making significant contributions to the industry. Collaborations and partnerships between established companies and startups are driving further advancements in AI technology.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research, such as advancements in reinforcement learning and computer vision, have been making headlines. The impact of AI on job markets and society is a topic of debate, with concerns about automation and job displacement. Regulations and policies are being developed to shape the future of AI, ensuring ethical and responsible AI deployment. Staying informed about these developments is essential for understanding the implications of AI on our daily lives.

## Conclusion

In conclusion, the future of Artificial Intelligence is filled with exciting possibilities and challenges. By keeping up with the latest trends, key players, and news in the AI industry, individuals can gain valuable insights into the evolving landscape of technology. It is essential to consider the ethical considerations in AI development and actively participate in discussions about the future of AI. As we navigate through the complexities of AI, let us embrace the opportunities it brings while being mindful of the responsibilities that come with it.

**Call to Action:** Explore the provided resources from Forbes, MIT Technology Review, and AI Trends for further learning. Share your thoughts and insights on the future of AI in the comments section below.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document:

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries.
- Mention the rapid advancements in AI technology and its potential for the future.

Key Points:
1. Latest Trends in AI:
- Machine learning and deep learning algorithms.
- Natural language processing and sentiment analysis.
- AI-powered chatbots and virtual assistants.
- Autonomous vehicles and robotics.
- A

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction to Artificial Intelligence
Artificial Intelligence (AI) has become a transformative force in various industries, revolutionizing the way businesses operate and enhancing the quality of our daily lives. With rapid advancements in AI technology, the possibilities for the future are limitless. From machine learning and deep learning algorithms to natural language processing and sentiment analysis, AI is reshaping how we interact with technology and each other.

## Latest Trends in AI
One of the key trends in AI is the rise of machine learning and deep learning algorithms, which enable computers to learn from data and improve their performance over time. Natural language processing and sentiment analysis have also gained prominence, allowing machines to understand and respond to human language in a more nuanced way. AI-powered chatbots and virtual assistant

In [14]:
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction to Artificial Intelligence
Artificial Intelligence (AI) has become a transformative force in various industries, revolutionizing the way businesses operate and enhancing the quality of our daily lives. With rapid advancements in AI technology, the possibilities for the future are limitless. From machine learning and deep learning algorithms to natural language processing and sentiment analysis, AI is reshaping how we interact with technology and each other.

## Latest Trends in AI
One of the key trends in AI is the rise of machine learning and deep learning algorithms, which enable computers to learn from data and improve their performance over time. Natural language processing and sentiment analysis have also gained prominence, allowing machines to understand and respond to human language in a more nuanced way. AI-powered chatbots and virtual assistants are becoming increasingly common, providing personalized and efficient customer service. Autonomous vehicles and robotics are pushing the boundaries of automation and transportation. In healthcare, AI is revolutionizing personalized medicine, offering tailored treatment plans based on individual patient data.

## Key Players in the AI Industry
Several tech giants are leading the way in AI innovation. Google, with its Google Brain and DeepMind projects, is at the forefront of AI research. Amazon's Alexa and Amazon Web Services are setting new standards for virtual assistants and cloud computing. IBM's Watson platform is a pioneer in cognitive computing, while Microsoft's Azure AI is making AI more accessible to businesses. Tesla's Autopilot technology is revolutionizing the automotive industry, and Apple's Siri and Core ML are bringing AI to the masses through mobile devices.

## Noteworthy News in AI
Recent breakthroughs in AI research have pushed the boundaries of what machines can achieve, from beating human champions in complex games to creating lifelike virtual environments. AI applications are increasingly being used to solve real-world problems, from predicting natural disasters to optimizing supply chains. Regulatory updates and ethical considerations in AI development are also gaining traction, with experts like Timnit Gebru advocating for responsible AI practices.

## Audience Q&A Section
**Q:** What are the future implications of AI on job markets?
**A:** While AI may automate certain tasks, it also creates new opportunities for jobs in AI development, data science, and technology management.

**Q:** How can individuals prepare for a career in AI?
**A:** Pursuing education in computer science, data analytics, and machine learning can help individuals enter the growing field of AI.

## Call to Action
As AI continues to shape the future of technology and business, it is essential for professionals, students, and enthusiasts to stay informed and explore the possibilities that AI offers. By attending AI conferences, exploring AI resources, and pursuing AI-related careers, individuals can be at the forefront of the AI revolution.

In conclusion, Artificial Intelligence is not just a technological advancement but a societal shift that has the potential to transform industries and improve the quality of life for individuals around the world. By staying updated on the latest trends and key players in the AI industry, we can harness the power of AI for positive change and innovation.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```